In [2]:
!pip install torchsummary
from torchsummary import summary

In [3]:
import os
import pandas as pd
import numpy as np
import pandas as pd
import time
import torch
import torch.nn as nn
from torch.nn.functional import one_hot
import torch.optim as optim 
from torch.optim import lr_scheduler
from torch import Tensor
from torch.utils.data import Dataset, DataLoader
from collections import Counter
# from transformers import AutoTokenizer, AutoModelForTokenClassification
# from transformers import pipeline
from gensim.utils import simple_preprocess
from string import punctuation
import wandb


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
def seed_all(seed_value):
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    
    if torch.cuda.is_available(): 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = True

seed_all(1)

In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [6]:
# batch_size = 1 #RNN
batch_size = 1 #CNN
learning_rate = 1e-04
display_step = 2500
epochs = 6
num_classes = 7
max_len = 75
pretrained_segmentation = True

In [7]:
# !ls -la /usr/lib/jvm/

In [8]:
!sudo apt install default-jre

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
default-jre is already the newest version (2:1.11-72build2).
0 upgraded, 0 newly installed, 0 to remove and 71 not upgraded.


In [9]:
!java -version

openjdk version "11.0.21" 2023-10-17
OpenJDK Runtime Environment (build 11.0.21+9-post-Ubuntu-0ubuntu122.04)
OpenJDK 64-Bit Server VM (build 11.0.21+9-post-Ubuntu-0ubuntu122.04, mixed mode, sharing)


In [10]:
!sudo apt-get --yes --force-yes install default-jdk 

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  default-jdk-headless libice-dev libpthread-stubs0-dev libsm-dev libx11-dev
  libxau-dev libxcb1-dev libxdmcp-dev libxt-dev openjdk-11-jdk
  openjdk-11-jdk-headless x11proto-dev xorg-sgml-doctools xtrans-dev
Suggested packages:
  libice-doc libsm-doc libx11-doc libxcb-doc libxt-doc openjdk-11-demo
  openjdk-11-source visualvm
The following NEW packages will be installed:
  default-jdk default-jdk-headless libice-dev libpthread-stubs0-dev libsm-dev
  libx11-dev libxau-dev libxcb1-dev libxdmcp-dev libxt-dev openjdk-11-jdk
  openjdk-11-jdk-headless x11proto-dev xorg-sgml-doctools xtrans-dev
0 upgraded, 15 newly installed, 0 to remove and 71 not upgraded.
Need to get 76.9 MB of archives.
After this operation, 90.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 openjdk-11-jdk-headless

In [11]:
!pip install py_vncorenlp
import py_vncorenlp
import os
if not os.path.exists('/kaggle/working/vncorenlp'):
    os.makedirs('/kaggle/working/vncorenlp')
    
py_vncorenlp.download_model(save_dir='/kaggle/working/vncorenlp') 
rdrsegmenter = py_vncorenlp.VnCoreNLP(annotators=["wseg"], save_dir='/kaggle/working/vncorenlp')

  Preparing metadata (setup.py) ... done
  Obtaining dependency information for pyjnius from https://files.pythonhosted.org/packages/f7/82/b438251543cb67dfbf24d2c973cedd68c85ba15c34dbee08ed6cfc41bf1a/pyjnius-1.6.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 17.2 MB/s eta 0:00:00a 0:00:01
  Created wheel for py_vncorenlp: filename=py_vncorenlp-0.1.4-py3-none-any.whl size=4306 sha256=a00c88381834d422a9ec2ae87e1afdd5d56a037f45ac237bb4f3d8b78ec7a0a6
  Stored in directory: /root/.cache/pip/wheels/d5/d9/bf/62632cdb007c702a0664091e92a0bb1f18a2fcecbe962d9827
Successfully built py_vncorenlp


--2023-12-15 16:45:16--  https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.2.jar
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 27412703 (26M) [application/octet-stream]
Saving to: ‘VnCoreNLP-1.2.jar’

     0K .......... .......... .......... .......... ..........  0%  704K 38s
    50K .......... .......... .......... .......... ..........  0% 4.18M 22s
   100K .......... .......... .......... .......... ..........  0% 1.99M 19s
   150K .......... .......... .......... .......... ..........  0% 6.18M 15s
   200K .......... .......... .......... .......... ..........  0% 7.11M 13s
   250K .......... .......... .......... .......... ..........  1% 9.79M 11s
   300K .......... .......... .......... .......... ..........  1% 2.86M 11s
   

2023-12-15 16:45:32 INFO  WordSegmenter:24 - Loading Word Segmentation model


In [12]:
def get_data(path):
    data = pd.read_excel(path, sheet_name=None)['Sheet1']
    data.columns = ['index', 'Emotion', 'Sentence'] 
    data.drop(columns=['index'], inplace=True)
    return data

train_data = get_data('/kaggle/input/uit-vsmec/UIT-VSMEC/train_nor_811.xlsx')
valid_data = get_data('/kaggle/input/uit-vsmec/UIT-VSMEC/valid_nor_811.xlsx')
test_data = get_data('/kaggle/input/uit-vsmec/UIT-VSMEC/test_nor_811.xlsx') 
 

In [13]:
# import torch
# import torch.nn as nn

# # FloatTensor containing pretrained weights
# weight = torch.FloatTensor([[1, 2.3, 3], [4, 5.1, 6.3]])
# embedding = nn.Embedding.from_pretrained(weight)
# # Get embeddings for index 1
# input = torch.LongTensor([0])
# embedding(input)

In [14]:
word2vec_path = '/kaggle/input/word2vec-300/word2vec_vi_words_300dims.txt'
file = open(word2vec_path, "r")
nlines, ebb_dim = file.readline().split() 
nlines, ebb_dim = int(nlines), int(ebb_dim)
word2vec_idx = dict()
word2vec_weights = list()
word2vec_weights.append([0 for j in range(ebb_dim)])
for i in range(1, nlines + 1):
    line = file.readline() 
    while line != "":  
        line = line.split()
        word = "_".join(line[0:len(line)-ebb_dim]).lower()
        if word not in word2vec_idx:
            word2vec_weights.append([float(j) for j in line[len(line)-ebb_dim:len(line)]]) 
            word2vec_idx[word] = i  
            break  
        line = file.readline()     
        
    if i%100000 == 0:
        print(i) 
        
# word2vec_weights.insert(0, [0 for j in range(dim)])        

100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000


In [15]:
# import torch
# torch.Tensor(word2vec_weights).to(device).shape

# print(len(word2vec_idx))

In [16]:
# # creating vocab:
# def vocab_clean_text(text):
#     text = " ".join(simple_preprocess(text)).strip()
#     if pretrained_segmentation == True:
#         res = tokenizing_nlp(text)
#         text = ""
#         for e in res:
#             if "##" in e["word"]:
#                 text = text + e["word"].replace("##","")
#             elif e["entity"] =="I":
#                 text = text + "_" + e["word"]
#             else:
#                 text = text + " " + e["word"] 
                
#     tokens = text.split()  
# #     table = str.maketrans('', '', punctuation.replace("_", ""))        
# #     tokens = [w.translate(table) for w in tokens] 
# #     tokens = [word for word in tokens if len(word) > 1]
#     if (max_len != -1):
#         tokens = tokens[0:max_len]     
#     return tokens

# def add_to_vocab(text, vocab):
#     tokens = vocab_clean_text(text)
#     vocab.update(tokens)
    
# def vocab_process_text(sentences, vocab):
#     count = 0
#     for sentence in sentences:
#         add_to_vocab(sentence, vocab)
#         count += 1
#         if count % 1000 == 0:
#             print(count)
            
# vocab = Counter()
# sentences = [x for x in train_data['Sentence']] 
# print(len(sentences))
# vocab_process_text(sentences, vocab) 

# # min_occurance = 2
# # vocab_list = [k for k,c in vocab.items() if c >= min_occurance]

# vocab = set(vocab_list)
# len(vocab)

In [17]:
dict_map = {
    "òa": "oà",
    "Òa": "Oà",
    "ÒA": "OÀ",
    "óa": "oá",
    "Óa": "Oá",
    "ÓA": "OÁ",
    "ỏa": "oả",
    "Ỏa": "Oả",
    "ỎA": "OẢ",
    "õa": "oã",
    "Õa": "Oã",
    "ÕA": "OÃ",
    "ọa": "oạ",
    "Ọa": "Oạ",
    "ỌA": "OẠ",
    "òe": "oè",
    "Òe": "Oè",
    "ÒE": "OÈ",
    "óe": "oé",
    "Óe": "Oé",
    "ÓE": "OÉ",
    "ỏe": "oẻ",
    "Ỏe": "Oẻ",
    "ỎE": "OẺ",
    "õe": "oẽ",
    "Õe": "Oẽ",
    "ÕE": "OẼ",
    "ọe": "oẹ",
    "Ọe": "Oẹ",
    "ỌE": "OẸ",
    "ùy": "uỳ",
    "Ùy": "Uỳ",
    "ÙY": "UỲ",
    "úy": "uý",
    "Úy": "Uý",
    "ÚY": "UÝ",
    "ủy": "uỷ",
    "Ủy": "Uỷ",
    "ỦY": "UỶ",
    "ũy": "uỹ",
    "Ũy": "Uỹ",
    "ŨY": "UỸ",
    "ụy": "uỵ",
    "Ụy": "Uỵ",
    "ỤY": "UỴ",
    }

def tone_normalize(text, dict_map):
    for i, j in dict_map.items():
        text = text.replace(i, j)
    return text

def preprocess_text(text):
    text = " ".join(simple_preprocess(text)).strip()
    
    output = rdrsegmenter.word_segment(text)
    text = " ".join(output)     
    
    text = tone_normalize(text, dict_map)  
    return text

def clean_text(text): 
    text = preprocess_text(text) 
    tokens = text.split() 
#     table = str.maketrans('', '', punctuation.replace("_", ""))
#     tokens = [w.translate(table) for w in tokens] 
    tokens = [w for w in tokens if w in word2vec_idx]
    if (max_len != -1):
        tokens = tokens[0:max_len]     
    tokens = ' '.join(tokens).strip()
    return tokens

In [18]:
def emotions_map(text):
        if text=='Enjoyment':
            return 0
        elif text=='Disgust':
            return 1
        elif text=='Sadness':
            return 2
        elif text=='Anger':
            return 3
        elif text=='Surprise':
            return 4
        elif text=='Fear':
            return 5
        else:
            return 6

In [19]:
train_data['Sentence'] = train_data['Sentence'].map(clean_text)
valid_data['Sentence'] = valid_data['Sentence'].map(clean_text)

train_data['Emotion'] = train_data['Emotion'].map(emotions_map)
valid_data['Emotion'] = valid_data['Emotion'].map(emotions_map)

In [20]:
train_data

,Emotion,Sentence
0,6,cho mình xin bài nhạc tên là gì với
1,1,cho đáng_đời con quỷ về nhà lôi con nhà mày ra...
2,1,lo học đi yêu_đương lol gì hay lại thích học_s...
3,0,uớc gì sau_này về già vẫn có_thể như cụ này
4,0,mỗi lần có video của con là cứ coi đi coi lại ...
...,...,...
5543,1,đường của nhà cụ hay sao mà cụ cấm người ta đỗ...
5544,6,nhìn mặt héo queo luôn
5545,6,tao đi xe_máy mỗi lần muốn để xe đi đâu là phi...
5546,0,thích thân_hình boss rồi nhan


In [21]:
counts = [1/(train_data['Emotion'].value_counts()[i]) for i in range(num_classes)]
weights = [i/(sum(counts)) for i in counts]
weights

[0.04773746415225396,
 0.0694444156388531,
 0.07853745422303239,
 0.19021731240207593,
 0.3073345832612053,
 0.23388355078368453,
 0.07284521953889489]

In [22]:
 !pip install pytorch-nlp 
from torchnlp.encoders.text import StaticTokenizerEncoder, stack_and_pad_tensors, pad_tensor

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.1/90.1 kB 2.3 MB/s eta 0:00:00a 0:00:01


In [23]:
max_len

75

In [24]:
# max_len = 50 

In [25]:
class DataClass(Dataset):
    def __init__(self, input_list, output_list, max_len = max_len, train=False): 
        super(DataClass, self).__init__()
        self.input_list = input_list
        self.output_list = output_list
        self.train = train
        self.max_len = max_len
 
        encoded_text = [torch.Tensor([word2vec_idx[word] for word in sentence.strip().split() if word in word2vec_idx] ).to(device) for sentence in self.input_list] 
#         self.input_list = [pad_tensor(x, length=max_len) for x in encoded_text] # CNN
        self.input_list = encoded_text # RNN
#         print(self.input_list[0].shape)
        self.output_list = [torch.tensor(int(y)) for y in self.output_list]
        self.output_list = [one_hot(y, num_classes = num_classes) for y in self.output_list]
        
    def __getitem__(self, index):   
        data = self.input_list[index]
        label = self.output_list[index]
       
        return data, label
    
    def __len__(self):
        return len(self.input_list)

In [26]:
train_input_list = [x for x in train_data["Sentence"]]
train_output_list = [x for x in train_data["Emotion"]]
train = [x for x in list(zip(train_input_list, train_output_list)) if len(x[0]) >0] 
train_input_list, train_output_list  = [list(t) for t in zip(*train)]

valid_input_list = [x for x in valid_data["Sentence"]]
valid_output_list = [x for x in valid_data["Emotion"]]
valid = [x for x in list(zip(valid_input_list, valid_output_list)) if len(x[0]) >0] 
valid_input_list, valid_output_list  = [list(t) for t in zip(*valid)]

In [27]:
train_input_list[0]

'cho mình xin bài nhạc tên là gì với'

In [28]:
train_set = DataClass(train_input_list, train_output_list)
valid_set = DataClass(valid_input_list, valid_output_list)

train_dataloader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
valid_dataloader = DataLoader(valid_set, batch_size=batch_size, shuffle=True)

In [29]:
# vocab_size = encoder.vocab_size 
# # vocab_size = torch.tensor(vocab_size).to(device).long()
# vocab_size

In [30]:
# class CNN(nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.embedding = nn.Embedding.from_pretrained(torch.Tensor(word2vec_weights).to(device), freeze = True)
#         self.conv1d_1 = nn.Conv1d(in_channels = ebb_dim, out_channels = 128, kernel_size = 3)
#         self.conv1d_2 = nn.Conv1d(in_channels = ebb_dim, out_channels = 128, kernel_size = 4)
#         self.conv1d_3 = nn.Conv1d(in_channels = ebb_dim, out_channels = 128, kernel_size = 5)
#         self.relu = nn.ReLU()
#         self.maxpool1d = nn.MaxPool1d(kernel_size = 2)
#         self.flatten = nn.Flatten()
#         self.drop_1 = nn.Dropout(p = 0.3)
#         self.drop_2 = nn.Dropout(p = 0.5)
 
#         self.linear_1 = nn.Linear(in_features = 13696, out_features = num_classes)
# #         self.linear_2 = nn.Linear(in_features = 20, out_features = num_classes)
#         self.sigmoid = nn.Sigmoid() 
#         self.softmax = nn.Softmax()
        
#     def forward(self, x): 
#         x = x.long() 
        
#         if torch.min(x) < 0:
#             print( torch.min(x), 'min')
        
#         if torch.max(x) >= len(word2vec_weights):
#             print( torch.max(x), 'max')
#             print(x, )
            
#         x = self.embedding(x) 
#         x = x.transpose(1, 2).contiguous() 
        
#         x_1 = self.conv1d_1(x) 
#         x_1 = self.relu(x_1)
#         x_1 = self.drop_1(x_1)
#         x_1 = self.maxpool1d(x_1)
#         x_1 = self.flatten(x_1)
        
#         x_2 = self.conv1d_2(x) 
#         x_2 = self.relu(x_2)
#         x_2 = self.drop_1(x_2)
#         x_2 = self.maxpool1d(x_2)
#         x_2 = self.flatten(x_2)
        
#         x_3 = self.conv1d_3(x) 
#         x_3 = self.relu(x_3)
#         x_3 = self.drop_1(x_3)
#         x_3 = self.maxpool1d(x_3)
#         x_3 = self.flatten(x_3)
# #         print(x_1.shape, x_2.shape, x_3.shape)
#         x = torch.cat( (x_1,x_2,x_3), dim = 1)
#         x = self.drop_2(x)
#         x = self.linear_1(x) 
# #         x = self.relu(x) 
# #         x = self.linear_2(x)
# #         x = self.sigmoid(x)
# #         x = self.softmax(x, dim = 1)
#         return x

In [31]:
# os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [32]:
# model = CNN() 
# model.to(device)
# torch.cuda.empty_cache() 
# summary(model, torch.Size([max_len]))

In [33]:
class Rnn(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(Rnn, self).__init__()
        self.embedding = nn.Embedding.from_pretrained(torch.Tensor(word2vec_weights).to(device))
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        x = x.long() 
        x = self.embedding(x) 
#         x = x.unsqueeze(0)
#         x = x.transpose(0,1).contiguous() 
#         print(x.shape)
        out, _ = self.rnn(x) 
        out = self.fc(out[:, -1, :])
        return out


In [50]:
model = Rnn(ebb_dim, 50, 7) 
model.to(device)
summary(model, torch.Size([max_len]))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
         Embedding-1              [-1, 75, 300]     437,513,400
               RNN-2  [[-1, 75, 50], [-1, 2, 50]]               0
            Linear-3                    [-1, 7]             357
Total params: 437,513,757
Trainable params: 357
Non-trainable params: 437,513,400
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 2.69
Params size (MB): 1668.98
Estimated Total Size (MB): 1671.67
----------------------------------------------------------------


In [35]:
def train(train_dataloader, 
          valid_dataloader,
          epoch, display_step):
    
    start_time = time.time()
    train_loss_epoch = 0
    valid_loss_epoch = 0
    train_correct = 0
    valid_correct = 0
    last_loss = 999999999  

    model.train()
    for i, (data,targets) in enumerate(train_dataloader):
        
        data, targets = data.to(device), targets.to(device) 
         
        optimizer.zero_grad()
        
        outputs = model(data) 
        
        loss = loss_function(outputs, torch.max(targets, dim = 1)[1])
        loss.backward()
        
        optimizer.step()
        
        train_correct += (torch.max(outputs, dim = 1)[1] == torch.max(targets, dim = 1)[1]).sum().float()

        train_loss_epoch += loss.item()  
        
        if (i+1) % display_step == 0:
            print('Train Epoch: {} [{}/{} ({}%)]\tLoss: {:.4f} '.format(
                epoch + 1, (i+1) * len(data), len(train_dataloader.dataset), 100 * (i+1) * len(data) / len(train_dataloader.dataset), loss.item() 
            ))     
                            
    train_loss_epoch/= len(train_set) 
    train_accuracy = train_correct / len(train_set) 
 
    model.eval()
    with torch.no_grad():
        for data, target in valid_dataloader:
            data, target = data.to(device), target.to(device)
            valid_output = model(data)
            valid_loss = nn.CrossEntropyLoss()(valid_output, target.float())    
            valid_loss_epoch += valid_loss.item()
            valid_correct += (torch.max(valid_output, dim = 1)[1] == torch.max(target, dim = 1)[1]).sum().float() 
              
    valid_loss_epoch/= len(valid_set)
    valid_accuracy = valid_correct / len(valid_set) 
        

    print(f"Done epoch #{epoch+1}, time for this epoch: {time.time()-start_time}s") 

    return train_loss_epoch , valid_loss_epoch, train_accuracy, valid_accuracy    

In [51]:
learning_rate = 1e-04

loss_function = nn.CrossEntropyLoss(torch.Tensor(weights).to(device))
# val_loss_function = 
optimizer = optim.AdamW(params=model.parameters(), lr=learning_rate )

In [37]:
# torch.Tensor(word2vec_weights).to(device).shape

In [38]:
# output = Tensor([[0, 0, 0, 0, 0, 1, 0]]).to(device)
# target = Tensor([[0, 0, 0, 0, 0, 1, 0]]).to(device)
# _, labels = torch.max(target, dim=1)
# print(labels)
# print(nn.CrossEntropyLoss()(output, labels))

In [52]:
epochs = 25
for epoch in range(epochs):
    train_loss_epoch = 0
    test_loss_epoch = 0 
    
    (train_loss_epoch, valid_loss_epoch, train_accuracy, valid_accuracy ) = train(train_dataloader, valid_dataloader, epoch, display_step)
    print(epoch+1, train_loss_epoch, train_accuracy , valid_loss_epoch, valid_accuracy )
    
# model.eval()
# test_loss_epoch = 0
# test_correct = 0
# with torch.no_grad():
#     for data, target in test_dataloader:
#         data, target = data.to(device), target.to(device)
#         test_output = model(data)
#         test_loss = nn.CrossEntropyLoss()(test_output, target.float())    
#         test_loss_epoch += test_loss.item()
#         test_correct += (torch.max(test_output, dim = 1)[1] == torch.max(target, dim = 1)[1]).sum().float() 

# test_loss_epoch/= len(test_set)
# test_accuracy = test_correct / len(test_set) 

# test_accuracy    

Train Epoch: 1 [2500/5545 (45.08566275924256%)]	Loss: 3.0442 
Train Epoch: 1 [5000/5545 (90.17132551848512%)]	Loss: 1.2950 
Done epoch #1, time for this epoch: 11.547150135040283s
1 1.774514088198556 tensor(0.2795, device='cuda:0') 1.6980377166333769 tensor(0.3207, device='cuda:0')
Train Epoch: 2 [2500/5545 (45.08566275924256%)]	Loss: 1.5705 
Train Epoch: 2 [5000/5545 (90.17132551848512%)]	Loss: 2.5143 
Done epoch #2, time for this epoch: 11.394279718399048s
2 1.6547273706433148 tensor(0.3410, device='cuda:0') 1.5798980239615843 tensor(0.4111, device='cuda:0')
Train Epoch: 3 [2500/5545 (45.08566275924256%)]	Loss: 1.3997 
Train Epoch: 3 [5000/5545 (90.17132551848512%)]	Loss: 1.0659 
Done epoch #3, time for this epoch: 11.615154027938843s
3 1.5612052063666775 tensor(0.3977, device='cuda:0') 1.5256489462086134 tensor(0.4052, device='cuda:0')
Train Epoch: 4 [2500/5545 (45.08566275924256%)]	Loss: 1.8389 
Train Epoch: 4 [5000/5545 (90.17132551848512%)]	Loss: 3.0542 
Done epoch #4, time for t

In [ ]:
#  Train Epoch: 1 [2500/5545 (45.08566275924256%)]	Loss: 2.6703 
# Train Epoch: 1 [5000/5545 (90.17132551848512%)]	Loss: 1.2856 
# Done epoch #1, time for this epoch: 11.433016061782837s
# 1 1.7866568134538325 tensor(0.2714, device='cuda:0') 1.7222807523808048 tensor(0.3120, device='cuda:0')
# Train Epoch: 2 [2500/5545 (45.08566275924256%)]	Loss: 1.6158 
# Train Epoch: 2 [5000/5545 (90.17132551848512%)]	Loss: 0.8576 
# Done epoch #2, time for this epoch: 11.454453945159912s
# 2 1.6782498595196456 tensor(0.3178, device='cuda:0') 1.6087347196072947 tensor(0.3688, device='cuda:0')
# Train Epoch: 3 [2500/5545 (45.08566275924256%)]	Loss: 1.7354 
# Train Epoch: 3 [5000/5545 (90.17132551848512%)]	Loss: 1.2501 
# Done epoch #3, time for this epoch: 11.278454542160034s
# 3 1.58172509663808 tensor(0.3675, device='cuda:0') 1.56503985704903 tensor(0.3980, device='cuda:0')
# Train Epoch: 4 [2500/5545 (45.08566275924256%)]	Loss: 2.2590 
# Train Epoch: 4 [5000/5545 (90.17132551848512%)]	Loss: 1.4715 
# Done epoch #4, time for this epoch: 11.230926752090454s
# 4 1.5077833250984616 tensor(0.4119, device='cuda:0') 1.5050952580510353 tensor(0.4257, device='cuda:0')
# Train Epoch: 5 [2500/5545 (45.08566275924256%)]	Loss: 2.0734 
# Train Epoch: 5 [5000/5545 (90.17132551848512%)]	Loss: 0.3611 
# Done epoch #5, time for this epoch: 11.406710386276245s
# 5 1.4520846662137614 tensor(0.4431, device='cuda:0') 1.4758948751076317 tensor(0.4271, device='cuda:0')
# Train Epoch: 6 [2500/5545 (45.08566275924256%)]	Loss: 1.7811 
# Train Epoch: 6 [5000/5545 (90.17132551848512%)]	Loss: 2.8625 
# Done epoch #6, time for this epoch: 11.332040309906006s
# 6 1.4075590718856688 tensor(0.4693, device='cuda:0') 1.423019446030737 tensor(0.4767, device='cuda:0')
# Train Epoch: 7 [2500/5545 (45.08566275924256%)]	Loss: 1.4728 
# Train Epoch: 7 [5000/5545 (90.17132551848512%)]	Loss: 1.1251 
# Done epoch #7, time for this epoch: 11.258015871047974s
# 7 1.3745132356143084 tensor(0.4830, device='cuda:0') 1.422888343062241 tensor(0.4548, device='cuda:0')
# Train Epoch: 8 [2500/5545 (45.08566275924256%)]	Loss: 1.6346 
# Train Epoch: 8 [5000/5545 (90.17132551848512%)]	Loss: 1.2246 
# Done epoch #8, time for this epoch: 11.371060132980347s
# 8 1.344347905567146 tensor(0.4967, device='cuda:0') 1.4299160635679873 tensor(0.4665, device='cuda:0')
# Train Epoch: 9 [2500/5545 (45.08566275924256%)]	Loss: 0.8983 
# Train Epoch: 9 [5000/5545 (90.17132551848512%)]	Loss: 1.6795 
# Done epoch #9, time for this epoch: 11.20369291305542s
# 9 1.3247415100472053 tensor(0.5082, device='cuda:0') 1.3995305583228523 tensor(0.4621, device='cuda:0')
# Train Epoch: 10 [2500/5545 (45.08566275924256%)]	Loss: 0.3760 
# Train Epoch: 10 [5000/5545 (90.17132551848512%)]	Loss: 1.3381 
# Done epoch #10, time for this epoch: 11.310073137283325s
# 10 1.30105892768427 tensor(0.5127, device='cuda:0') 1.4008449894748085 tensor(0.4796, device='cuda:0')
# Train Epoch: 11 [2500/5545 (45.08566275924256%)]	Loss: 0.3494 
# Train Epoch: 11 [5000/5545 (90.17132551848512%)]	Loss: 0.6700 
# Done epoch #11, time for this epoch: 11.23579740524292s
# 11 1.2832327672551622 tensor(0.5217, device='cuda:0') 1.4296831918646833 tensor(0.4738, device='cuda:0')
# Train Epoch: 12 [2500/5545 (45.08566275924256%)]	Loss: 0.5925 
# Train Epoch: 12 [5000/5545 (90.17132551848512%)]	Loss: 0.4240 
# Done epoch #12, time for this epoch: 11.216334581375122s
# 12 1.2628835731712518 tensor(0.5326, device='cuda:0') 1.3862142076786423 tensor(0.4767, device='cuda:0')
# Train Epoch: 13 [2500/5545 (45.08566275924256%)]	Loss: 1.7210 
# Train Epoch: 13 [5000/5545 (90.17132551848512%)]	Loss: 2.9751 
# Done epoch #13, time for this epoch: 11.249133110046387s
# 13 1.2465822902984238 tensor(0.5389, device='cuda:0') 1.4082583839035765 tensor(0.4606, device='cuda:0')
# Train Epoch: 14 [2500/5545 (45.08566275924256%)]	Loss: 2.2272 
# Train Epoch: 14 [5000/5545 (90.17132551848512%)]	Loss: 1.1075 
# Done epoch #14, time for this epoch: 11.237936973571777s
# 14 1.2311285392088165 tensor(0.5432, device='cuda:0') 1.4045106111523362 tensor(0.4825, device='cuda:0')
# Train Epoch: 15 [2500/5545 (45.08566275924256%)]	Loss: 1.3527 
# Train Epoch: 15 [5000/5545 (90.17132551848512%)]	Loss: 0.5960 
# Done epoch #15, time for this epoch: 11.240567684173584s
# 15 1.2204175288826764 tensor(0.5464, device='cuda:0') 1.3711980169410913 tensor(0.4898, device='cuda:0')
# Train Epoch: 16 [2500/5545 (45.08566275924256%)]	Loss: 1.1701 
# Train Epoch: 16 [5000/5545 (90.17132551848512%)]	Loss: 0.9051 
# Done epoch #16, time for this epoch: 11.306766033172607s
# 16 1.2072136223893966 tensor(0.5493, device='cuda:0') 1.364149249859328 tensor(0.4898, device='cuda:0')
# Train Epoch: 17 [2500/5545 (45.08566275924256%)]	Loss: 2.6042 
# Train Epoch: 17 [5000/5545 (90.17132551848512%)]	Loss: 1.4800 
# Done epoch #17, time for this epoch: 11.244050979614258s
# 17 1.1984903046570725 tensor(0.5470, device='cuda:0') 1.3899994983443713 tensor(0.4708, device='cuda:0')
# Train Epoch: 18 [2500/5545 (45.08566275924256%)]	Loss: 1.0684 
# Train Epoch: 18 [5000/5545 (90.17132551848512%)]	Loss: 2.8669 
# Done epoch #18, time for this epoch: 11.256582021713257s
# 18 1.181341670501752 tensor(0.5594, device='cuda:0') 1.3692503971653196 tensor(0.4781, device='cuda:0')
# Train Epoch: 19 [2500/5545 (45.08566275924256%)]	Loss: 1.3475 
# Train Epoch: 19 [5000/5545 (90.17132551848512%)]	Loss: 0.8791 
# Done epoch #19, time for this epoch: 11.271088600158691s
# 19 1.1750014459029028 tensor(0.5621, device='cuda:0') 1.3662952640210617 tensor(0.4796, device='cuda:0')
# Train Epoch: 20 [2500/5545 (45.08566275924256%)]	Loss: 2.1329 
# Train Epoch: 20 [5000/5545 (90.17132551848512%)]	Loss: 0.9636 
# Done epoch #20, time for this epoch: 11.114390134811401s
# 20 1.1627698885421356 tensor(0.5697, device='cuda:0') 1.3670324770701132 tensor(0.4694, device='cuda:0')
# Train Epoch: 1 [2500/5545 (45.08566275924256%)]	Loss: 1.2782 
# Train Epoch: 1 [5000/5545 (90.17132551848512%)]	Loss: 1.9108 
# Done epoch #1, time for this epoch: 11.384056568145752s
# 1 1.1517097267129932 tensor(0.5722, device='cuda:0') 1.3536576927930073 tensor(0.4781, device='cuda:0')
# Train Epoch: 2 [2500/5545 (45.08566275924256%)]	Loss: 1.0932 
# Train Epoch: 2 [5000/5545 (90.17132551848512%)]	Loss: 0.4131 
# Done epoch #2, time for this epoch: 11.563904285430908s
# 2 1.1472793414455655 tensor(0.5766, device='cuda:0') 1.332399787386722 tensor(0.4927, device='cuda:0')
# Train Epoch: 3 [2500/5545 (45.08566275924256%)]	Loss: 0.3389 
# Train Epoch: 3 [5000/5545 (90.17132551848512%)]	Loss: 1.4740 
# Done epoch #3, time for this epoch: 11.454102039337158s
# 3 1.1394104073282643 tensor(0.5776, device='cuda:0') 1.3768546853931323 tensor(0.4869, device='cuda:0')
# Train Epoch: 4 [2500/5545 (45.08566275924256%)]	Loss: 0.6976 
# Train Epoch: 4 [5000/5545 (90.17132551848512%)]	Loss: 2.4555 
# Done epoch #4, time for this epoch: 11.603801250457764s
# 4 1.132159784036068 tensor(0.5825, device='cuda:0') 1.3176976599602936 tensor(0.4985, device='cuda:0')
# Train Epoch: 5 [2500/5545 (45.08566275924256%)]	Loss: 2.5939 
# Train Epoch: 5 [5000/5545 (90.17132551848512%)]	Loss: 1.1063 
# Done epoch #5, time for this epoch: 11.44542670249939s
# 5 1.1230132907613057 tensor(0.5845, device='cuda:0') 1.3561551527856 tensor(0.5000, device='cuda:0')
# Train Epoch: 1 [2500/5545 (45.08566275924256%)]	Loss: 0.5810 
# Train Epoch: 1 [5000/5545 (90.17132551848512%)]	Loss: 0.1977 
# Done epoch #1, time for this epoch: 11.543864011764526s
# 1 1.1120709208236643 tensor(0.5863, device='cuda:0') 1.340247570830275 tensor(0.5102, device='cuda:0')
# Train Epoch: 2 [2500/5545 (45.08566275924256%)]	Loss: 2.9522 
# Train Epoch: 2 [5000/5545 (90.17132551848512%)]	Loss: 5.3727 
# Done epoch #2, time for this epoch: 11.494173288345337s
# 2 1.1104150114359705 tensor(0.5881, device='cuda:0') 1.3375107721851884 tensor(0.5102, device='cuda:0')
# Train Epoch: 3 [2500/5545 (45.08566275924256%)]	Loss: 0.0524 
# Train Epoch: 3 [5000/5545 (90.17132551848512%)]	Loss: 1.6598 
# Done epoch #3, time for this epoch: 11.579136610031128s
# 3 1.102625635948714 tensor(0.5942, device='cuda:0') 1.34025338221526 tensor(0.5015, device='cuda:0')

In [41]:
test_data['Sentence'] = test_data['Sentence'].map(clean_text) 
test_data['Emotion'] = test_data['Emotion'].map(emotions_map)

test_output_list = [x for x in test_data["Emotion"]]
test_input_list = [x for x in test_data["Sentence"]]
test = [x for x in list(zip(test_input_list, test_output_list)) if len(x[0]) >0] 
test_input_list, test_output_list  = [list(t) for t in zip(*test)]

test_set = DataClass(test_input_list, test_output_list) 
test_dataloader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

In [53]:
model.eval()
test_loss_epoch = 0
test_correct = 0
predicts = list()
outputs = list()
with torch.no_grad():
    for data, target in test_dataloader:
        data, target = data.to(device), target.to(device)
        test_output = model(data)
        test_loss = loss_function(test_output, target.float())    
        test_loss_epoch += test_loss.item()
        test_correct += (torch.max(test_output, dim = 1)[1] == torch.max(target, dim = 1)[1]).sum().float() 
        predicts.append(torch.max(test_output, dim = 1)[1].item())
        outputs.append(torch.max(target, dim = 1)[1].item()) 

test_loss_epoch/= len(test_set)
test_accuracy = test_correct / len(test_set) 

# print(test_accuracy)
print()
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
print(classification_report(predicts, outputs, digits = 4)) 



              precision    recall  f1-score   support

           0     0.5885    0.5567    0.5722       203
           1     0.6591    0.4628    0.5437       188
           2     0.6638    0.4667    0.5480       165
           3     0.1750    0.5385    0.2642        13
           4     0.1622    0.3529    0.2222        17
           5     0.5217    0.7742    0.6234        31
           6     0.2403    0.4133    0.3039        75

    accuracy                         0.4986       692
   macro avg     0.4301    0.5093    0.4397       692
weighted avg     0.5667    0.4986    0.5175       692



In [ ]:
# from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
# print(classification_report(predicts, outputs))

In [55]:
def save_model(model, optimizer, path):
    checkpoint = {
        "model": model.state_dict(),
        "optimizer": optimizer.state_dict(),
    }
    torch.save(checkpoint, path)
    print('model saved')
    
save_model(model, optimizer, '/kaggle/working/model_1.pth')    

model saved
